# Recap: Training a Conv Model
### Detecting Lines of Three on a Board of 7x7

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf

from domoku.tools import GomokuTools as Gt

In [3]:
input_size=7

---

## Sampling Random Boards

In [4]:
from notebooks.ml_basics_recap.data import new_sample
sample = new_sample(board_size=input_size, num_blacks=20, num_whites=0)
sample.shape

(1, 7, 7, 2)

In [5]:
Gt.print_bin(sample, True)

shape: (7, 7, 2)
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 1. 0. 0.]
 [1. 0. 0. 1. 1. 0. 1.]]


---

## The Detection Map

In [6]:
from notebooks.ml_basics_recap.models.heuristic_detector import HeuristicDetector
detector = HeuristicDetector(input_size)

2022-05-26 20:57:06.041101: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
tf.floor(detector(sample)+.01)

<tf.Tensor: shape=(1, 7, 7), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)>

---

## Batches From a Dataset

In [8]:
BATCH_SIZE=2

In [9]:
from notebooks.ml_basics_recap.data import new_sample, new_dataset
dataset = new_dataset(100, lambda: new_sample(input_size, 20, 0), detector).batch(BATCH_SIZE)

In [10]:
iterator  = iter(dataset)
states, labels = iterator.next()
labels.shape

TensorShape([2, 1, 7, 7])

In [11]:
states.shape

TensorShape([2, 1, 7, 7, 2])

---

## The Trainable Model

In [12]:
from notebooks.ml_basics_recap.models import SimpleConvQFunction

model_q = SimpleConvQFunction(input_size, n_filters=8, n_layers=4)

In [13]:
print(model_q(states).shape)
model_q(states).shape

(2, 1, 7, 7)


TensorShape([2, 1, 7, 7])

---

## Training

In [31]:
TRAIN_SIZE = 1024 * 8
TEST_SIZE = 1024
BATCH_SIZE = 1024

### Create a dataset from the heuristics

In [32]:
train_dataset = new_dataset(size=TRAIN_SIZE,
                            sampler=lambda: new_sample(board_size=input_size, num_blacks=20, num_whites=0),
                            labeler=detector
                            ).batch(BATCH_SIZE)
test_dataset = new_dataset(size=TEST_SIZE,
                           sampler=lambda: new_sample(board_size=input_size, num_blacks=20, num_whites=0),
                           labeler=detector
                           ).batch(BATCH_SIZE)

In [37]:
from notebooks.ml_basics_recap.training import Trainer

trainer = Trainer(model_q, train_data=train_dataset, test_data=test_dataset)
trainer.train(20)

Epoch 1, Loss: 0.030457723885774612, Accuracy: 0.17452141642570496,     Test Loss: 0.02067066915333271, Test Accuracy: 0.14377297461032867
Epoch 2, Loss: 0.008607793599367142, Accuracy: 0.09277819097042084,     Test Loss: 0.007714968174695969, Test Accuracy: 0.0878349021077156
Epoch 3, Loss: 0.005864743143320084, Accuracy: 0.07658161967992783,     Test Loss: 0.002198312897235155, Test Accuracy: 0.04688616469502449
Epoch 4, Loss: 0.002155597321689129, Accuracy: 0.04642840847373009,     Test Loss: 0.0019296678947284818, Test Accuracy: 0.043927986174821854
Epoch 5, Loss: 0.001383040682412684, Accuracy: 0.03718925639986992,     Test Loss: 0.0009438393753953278, Test Accuracy: 0.030721968039870262
Epoch 6, Loss: 0.000784075993578881, Accuracy: 0.02800135873258114,     Test Loss: 0.0006398530676960945, Test Accuracy: 0.02529531717300415
Epoch 7, Loss: 0.00047482206718996167, Accuracy: 0.021790411323308945,     Test Loss: 0.00037889863597229123, Test Accuracy: 0.019465317949652672
Epoch 8, Lo

---

## Manual Inspection

In [38]:
BATCH_SIZE=1

In [39]:
dataset = new_dataset(10, lambda: new_sample(input_size, 20, 0), detector).batch(BATCH_SIZE)
iterator  = iter(dataset)

### Labels vs Learned

In [42]:
states, labels = iterator.next()
pred = model_q(states)

print(tf.floor(labels+.01))
print(np.round(pred, 2))

tf.Tensor(
[[[[0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 1. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 1. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]]]], shape=(1, 1, 7, 7), dtype=float32)
[[[[ 0.   -0.   -0.   -0.    0.    0.    0.  ]
   [-0.   -0.   -0.    1.   -0.    0.    0.  ]
   [-0.   -0.    0.    0.    0.    0.    0.  ]
   [-0.02  1.   -0.    0.   -0.    0.    0.  ]
   [-0.01 -0.01 -0.    0.    0.    0.    0.  ]
   [-0.02 -0.01 -0.    0.    0.    0.    0.  ]
   [ 0.    0.    0.    0.    0.    0.    0.  ]]]]
